In [11]:
import duckdb
from datetime import datetime, timedelta
import pandas as pd

# Get yesterday's date
yesterday = datetime.today() - timedelta(days=1)

# Manually format the file name and db-name
yesterday_file = f"data/{yesterday.month}_{yesterday.day}_{yesterday.year}_A.csv"
yesterday_db = f"data/{yesterday.month}_{yesterday.day}_{yesterday.year}_A.db"

# Read the CSV file into a DataFrame
df = pd.read_csv(yesterday_file)

# Ensure column names are correct and strip any extra spaces
df.columns = df.columns.str.strip()

# Check if 'Numbers' is the correct column name
if 'Numbers' not in df.columns:
    raise KeyError(f"'Numbers' column not found. Available columns are: {df.columns}")

# Split the 'Numbers' column into 5 separate columns
df[['num1', 'num2', 'num3', 'num4', 'num5']] = df['Numbers'].str.split(' ', expand=True).astype(int)

# Drop the original 'Numbers' column as it's no longer needed
df.drop(columns=['Numbers'], inplace=True)

# Connect to DuckDB and create a permanent table
con = duckdb.connect('yesterday_db')

# Create the table if it doesn't exist
con.execute('''
CREATE TABLE IF NOT EXISTS numbers_data (
    Date TEXT PRIMARY KEY,
    num1 INTEGER,
    num2 INTEGER,
    num3 INTEGER,
    num4 INTEGER,
    num5 INTEGER
)
''')

# Insert the DataFrame data into the DuckDB table
con.execute("INSERT INTO numbers_data SELECT * FROM df")

# Close the DuckDB connection
con.close()